In [2]:
import pandas as pd
import numpy as np

data_path = '../data/'

In [3]:
!ls ../data/

gender_submission.csv  test.csv  train.csv


In [4]:
train = pd.read_csv(data_path + 'train.csv')
test = pd.read_csv(data_path + 'test.csv')
sample_submission = pd.read_csv(data_path + 'gender_submission.csv')

In [25]:
sample_submission.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [5]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [10]:
train.groupby(by=['Survived']).mean()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
Survived,,,,,,
0,447.016393,2.531876,30.626179,0.553734,0.329690,22.117887
1,444.368421,1.950292,28.343690,0.473684,0.464912,48.395408


In [11]:
print(train['Age'].max())
print(train['Age'].min())

80.0
0.42


In [14]:
train.loc[train['Age'] == train['Age'].min()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
803,804,1,3,"Thomas, Master. Assad Alexander",male,0.42,0,1,2625,8.5167,NaN,C


In [23]:
train.groupby('Sex').sum()[['Survived']] 

,Survived
Sex,
female,233
male,109


In [24]:
train.groupby('Embarked').sum()[['Survived']]

,Survived
Embarked,
C,93
Q,30
S,217


In [142]:
reduced_train = train[['Survived', 'Embarked', 'Sex', 'Age', 'Fare']].copy()
reduced_train.head()

,Survived,Embarked,Sex,Age,Fare
0,0,S,male,22.0,7.2500
1,1,C,female,38.0,71.2833
2,1,S,female,26.0,7.9250
3,1,S,female,35.0,53.1000
4,0,S,male,35.0,8.0500


In [143]:
age_mean = reduced_train['Age'].mean()
age_max = reduced_train['Age'].max()
fare_max = reduced_train['Fare'].max()
fare_mean = reduced_train['Fare'].mean()
reduced_train.loc[:, 'Age'] = reduced_train.loc[:, 'Age'].fillna(age_mean) / age_max
reduced_train.loc[:, 'Fare'] = reduced_train.loc[:, 'Fare'] / fare_max
reduced_train.head()

,Survived,Embarked,Sex,Age,Fare
0,0,S,male,0.2750,0.014151
1,1,C,female,0.4750,0.139136
2,1,S,female,0.3250,0.015469
3,1,S,female,0.4375,0.103644
4,0,S,male,0.4375,0.015713


In [144]:
reduced_train.loc[reduced_train['Sex'] == 'male', 'Sex'] = 0
reduced_train.loc[reduced_train['Sex'] == 'female', 'Sex'] = 1

reduced_train.head()

,Survived,Embarked,Sex,Age,Fare
0,0,S,0,0.2750,0.014151
1,1,C,1,0.4750,0.139136
2,1,S,1,0.3250,0.015469
3,1,S,1,0.4375,0.103644
4,0,S,0,0.4375,0.015713


In [145]:
processed_train = pd.concat([reduced_train, pd.get_dummies(reduced_train['Embarked'])], axis=1).drop('Embarked', axis=1)

X = processed_train.drop('Survived', axis = 1).copy()
y = processed_train['Survived'].copy()

In [146]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=13)
model = RidgeClassifier(alpha=1.0)

In [147]:
model.fit(X_train, y_train)

RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
        max_iter=None, normalize=False, random_state=None, solver='auto',
        tol=0.001)

In [148]:
from sklearn.metrics import accuracy_score

In [149]:
predictions = model.predict(X_train)
print(accuracy_score(y_true=y_train, y_pred=predictions))

predictions = model.predict(X_test)
print(accuracy_score(y_true=y_test, y_pred=predictions))

0.790268456375839
0.7830508474576271


In [150]:
X_train.columns

Index(['Sex', 'Age', 'Fare', 'C', 'Q', 'S'], dtype='object')

In [151]:
model = RidgeClassifier(alpha=1.0)
model.fit(X, y)

RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
        max_iter=None, normalize=False, random_state=None, solver='auto',
        tol=0.001)

In [153]:
test.head()

reduced_test = test[['Embarked', 'Sex', 'Age', 'Fare']].copy()


reduced_test.loc[:, 'Age'] = reduced_test.loc[:, 'Age'].fillna(age_mean) / age_max
reduced_test.loc[:, 'Fare'] = reduced_test.loc[:, 'Fare'].fillna(fare_mean) / fare_max
reduced_test.loc[reduced_test['Sex'] == 'male', 'Sex'] = 0
reduced_test.loc[reduced_test['Sex'] == 'female', 'Sex'] = 1
X_test = pd.concat([reduced_test, pd.get_dummies(reduced_test['Embarked'])], axis=1).drop('Embarked', axis=1)

In [154]:
predictions = model.predict(X_test)

In [155]:
predictions

array([0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [160]:
pd.Series?

In [161]:
submission = pd.concat([test[['PassengerId']].copy(), pd.Series(predictions, name='Survived')], axis=1)
submission.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [164]:
submission.to_csv(data_path + 'submission_v0.csv', index=False)